In [13]:
# # Librairies
import os
import pandas as pd
from argparse import ArgumentParser
import sys
import fnmatch
import glob
import numpy as np


# recupérer le nombre de variant optimal du fichier haplo
def optimal_variant(haplo_file):
    numb_OptVar= int(haplo_file.split("_")[1].replace("Variant.csv",''))
    return numb_OptVar


# recupérer les variants majoritaires de chaque fichier haplo
def maj_variants(haplo_file):

    read_file=pd.read_csv(f"{gene_dir}/{haplo_file}", index_col=[0])
    var_pos = read_file["Position"]
    read_file= read_file.drop(columns=["Position"])
    read_file.columns= range(1, len(read_file.columns)+1)
    variants=[]
    for col in read_file.columns:
        variant= read_file[col].tolist()
        variants.append(variant)
    return variants, var_pos


# print("****")
nb_replicate=10
seuil_replicate=8

freq_dir= "/home/osidibe/work/fluGenAvi_fugace/freq_allelic_OSI"

directory = "/home/osidibe/work/fluGenAvi_fugace/Analyse_postDESMAN_P2/DESMAN_iterAddPost"
gene = "blaTEM_1A_1_HM749966"

gene_dir = os.path.join(directory + "/DESMAN_" + gene)

nb_haplo=0
for haplo_file in os.listdir(gene_dir):
    

    if fnmatch.fnmatch(haplo_file, "haplo_*"):

        variants , var_pos = maj_variants(haplo_file)
    

        """ Step_1 : ne pas analyser les fichiers de variants incomplets """

        if (optimal_variant(haplo_file)*int(nb_replicate)) == len(variants):
            # print(f"{haplo_file} bon nombre de variant")
            # print(len(variants))


            """ Step_2 : Garder les haplotypes présents au moins dans 80% des réplicats  """
        
            variants_not_dup = [list(x) for x in set(tuple(x) for x in variants)]
        
            variant_file=pd.DataFrame(var_pos)

            for variant in variants_not_dup:
                # print(variant)
                nb_dup=0
                for var in variants:
                    # print(var)
                    if variant == var:
                        nb_dup+=1
                
                if nb_dup >= seuil_replicate:
                    # variant = pd.Series(variant, index= var_pos)
                    variant_file =pd.concat([variant_file, pd.Series(variant)], axis=1)
                    variant_file.to_csv(f"{gene_dir}/{haplo_file}")
                    
            # print(variant_file)
            nb_haplo+=1

        
        else :
            os.remove(f"{gene_dir}/{haplo_file}")

""" Step_3 : créer un fichier des variants uniques et majoritaires  """

final_variant=pd.DataFrame()

exit_loop = False

for i in range(1, nb_haplo+1):
    j= i+1

    haplo_file_1= f"haplo_{i}Variant.csv"
    haplo_file_2= f"haplo_{j}Variant.csv"

    try:
        variants_1, var_pos_1 = maj_variants(haplo_file_1)
        # print(variants_1)
        variants_2, var_pos_2 = maj_variants(haplo_file_2)
        # print(variants_2)

        commun_var=[]
        for var in variants_1:
            if var in variants_2:
                exit_loop = False


            else:
                for var1 in variants_1:
                    final_variant = pd.concat([final_variant, pd.Series(var1)], axis=1)
                    final_variant.to_csv(f"{gene_dir}/final_variant.csv")
                exit_loop = True
                break
                


    except FileNotFoundError :
        for var1 in variants_1:
            final_variant = pd.concat([final_variant, pd.Series(var1)], axis=1)
            final_variant.to_csv(f"{gene_dir}/final_variant.csv")
        exit_loop = True

        
    if exit_loop:
        break
        
print(final_variant)

    

""" Step_4: vérifier si les variants majoritaires sont correctements reconstruits  """

# Ouvrir les tables de comptages
freq_repository = glob.glob(f"{freq_dir}/*")
final_variant.columns = range(1, len(final_variant.columns)+1)

nb_var=0
for col in final_variant.columns:

    nb_base=0
    index_var=0
    for base in final_variant[col].tolist():

        for freq in freq_repository:
            try:
                comptage = pd.read_csv(f"{freq}/{gene}.freq", sep="\t", index_col=[0])

                if var_pos[index_var] in comptage.index:
                    print(var_pos[index_var], base)
                    if comptage.loc[var_pos[index_var],base] != 0:
                        nb_base+=1
                        index_var+=1
                        break
                    else :
                        print(f"{base} not correct")
                    
            except FileNotFoundError :
                print(f"{freq}/{gene}.freq not exist or position not exit in file")

    if nb_base == len(final_variant.index):
        print(final_variant[col])
        nb_var+=1
        

    else :
        final_variant=final_variant.drop(columns=col, axis=1)

if nb_var == len(final_variant.columns):
    # print("All variant correct")
    final_variant.to_csv(f"{gene_dir}/final_variant.csv")
    print(final_variant)
   


   0  0
0  T  C
1  A  A
2  T  C
3  T  G
4  T  T
5  T  T
6  C  C
7  G  G
/home/osidibe/work/fluGenAvi_fugace/freq_allelic_OSI/CTU_ACI/blaTEM_1A_1_HM749966.freq not exist or position not exit in file
/home/osidibe/work/fluGenAvi_fugace/freq_allelic_OSI/CTU_ACJ/blaTEM_1A_1_HM749966.freq not exist or position not exit in file
/home/osidibe/work/fluGenAvi_fugace/freq_allelic_OSI/CTU_ACK/blaTEM_1A_1_HM749966.freq not exist or position not exit in file
/home/osidibe/work/fluGenAvi_fugace/freq_allelic_OSI/CTU_ACL/blaTEM_1A_1_HM749966.freq not exist or position not exit in file
/home/osidibe/work/fluGenAvi_fugace/freq_allelic_OSI/CTU_ACM/blaTEM_1A_1_HM749966.freq not exist or position not exit in file
/home/osidibe/work/fluGenAvi_fugace/freq_allelic_OSI/CTU_ACN/blaTEM_1A_1_HM749966.freq not exist or position not exit in file
/home/osidibe/work/fluGenAvi_fugace/freq_allelic_OSI/CTU_ACO/blaTEM_1A_1_HM749966.freq not exist or position not exit in file
/home/osidibe/work/fluGenAvi_fugace/freq_allel

In [80]:
""" Step_4: vérifier si les variants majoritaires sont correctements reconstruits  """

# Ouvrir les tables de comptages
freq_repository = glob.glob(f"{freq_dir}/*")
nb_var=0
for col in range(1, len(final_variant.columns)+1):

    nb_base=0
    index_var = 0

    for base in final_variant[col].tolist():

        for freq in freq_repository:
            try:
                comptage = pd.read_csv(f"{freq}/{gene}.freq", sep="\t", index_col=[0])

            except FileNotFoundError :
                print(f"{freq}/{gene}.freq not Found")


            if comptage.loc[var_pos[index_var],base] != 0 :
                # print(f"{base} is correct")
                nb_base+=1
                index_var+=1
                break

    if nb_base == len(final_variant.index):
        # print(f"{final_variant[col]} is correct variant")
        nb_var+=1

    else :
        final_variant=final_variant.drop(final_variant[col])
        # print(f"{final_variant[col]} is incorrect variant")

if nb_var == len(final_variant.columns):
    print("All variant correct")

final_variant.to_csv(f"{gene_dir}/final_variant.csv")




All variant correct
